# 导入数据

In [1]:
from yz.task_dec2high_pddl import *

In [2]:
args = set_up_args()

In [3]:
args.gpu = True

In [4]:
task2plan_train = load_task_and_plan_json(args, "train")

{'tests_seen': 1533, 'tests_unseen': 1529, 'train': 21023, 'valid_seen': 820, 'valid_unseen': 821}


In [5]:
tpd = TaskPlanDataset(task2plan_train)

# 写入字典

In [6]:
#tpd[np.random.randint(1000)]

In [7]:
tpd.write_vocab("yz/vocab.txt")

# 训练

In [8]:
train_dataloader = DataLoader(tpd, batch_size=args.batch, shuffle=True,)

In [9]:
tasks, plans = next(iter(train_dataloader))

In [10]:
tasks

('Move the pump bottle from next to the sink to inside the cabinet. turn right, then turn right again and move to the sinks. pick up the pump bottle to the left of the rightmost sink. turn around and take a step forward, then turn back around to face the sinks again. open the right cabinet door on the rightmost cabinet under the sink and put the pump bottle inside, then close the door. ',
 'Get box of tissues out of drawer and turn on lamp. move slightly right and proceed to dresser. open bottom right drawer of dresser, remove box of tissues, and close drawer. turn left, proceed to wall.  turn right, proceed to dresser. turn on lamp on top of dresser. ',
 'Place the box with the credit card on the leather chair. turn around towards the other side of the room and look up to see the vase on the upper shelf. grab the credit card next to the vase. turn right, head towards the coffee table, stand in front of the opened box. place the card in the box. pick up the box with the card in it from

In [11]:
from transformers import BertTokenizer

In [12]:
# Load pre-trained model tokenizer (vocabulary)
encoder_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# tokenized_text = encoder_tokenizer(tasks, padding=True, truncation=True, max_length=100, return_tensors="pt").input_ids

decoder_tokenizer = BasicTokenizer("yz/vocab.txt")

#targets = torch.LongTensor(decoder_tokenizer.tokenize(plans))

In [13]:
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel

# 导入模型 BERT

In [14]:
# Initializing a BERT bert-base-uncased style configuration
config_encoder = BertConfig()
config_decoder = BertConfig()

config_decoder.update({
    "vocab_size": len(decoder_tokenizer.vocab),
    "num_hidden_layers":3,
    "num_attention_heads":3
})

config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)

In [15]:
model = EncoderDecoderModel(config=config)

In [16]:
model.encoder = BertModel.from_pretrained('bert-base-uncased')

In [17]:
if args.gpu and torch.cuda.is_available():
    model = model.cuda()

In [18]:
loss_fun = nn.CrossEntropyLoss(ignore_index=0)

In [19]:
optimizer = torch.optim.Adam(model.decoder.parameters(), lr=args.lr)

In [20]:
from datetime import datetime

# datetime object containing current date and time
begin_time = datetime.now()
print(begin_time)

2021-03-15 20:27:12.884322


In [ ]:
for epoch in range(args.epoch):
    model.train()
    train_loss_list = []
    train_acc_list = []
    for tasks, plans in tqdm(train_dataloader):
        tokenized_text = encoder_tokenizer(tasks, padding=True, truncation=True, 
                                           max_length=100, return_tensors="pt").input_ids
#         try:
        targets = torch.LongTensor(decoder_tokenizer.tokenize(plans))
#         except:
#             print(tasks)
#             print(plans)
        if args.gpu and torch.cuda.is_available():
            tokenized_text = tokenized_text.to("cuda")
            targets = targets.to("cuda")
        
        outputs = model(input_ids = tokenized_text, decoder_input_ids = targets[:,:-1])
        
        output_dim = outputs.logits.shape[-1]
        outputs.logits = outputs.logits.contiguous().view(-1, output_dim)
        labels = targets[:,1:].contiguous().view(-1)
        
        train_loss = loss_fun(outputs.logits, labels)
        #print(train_loss)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        train_acc = calculate_accuracy(outputs.logits, labels)
        
        train_loss_list.append(train_loss.item())
        train_acc_list.append(train_acc)
        
        
    mean_train_loss = sum(train_loss_list) / (len(train_loss_list) + 1e-4)
    mean_train_acc = sum(train_acc_list) / (len(train_acc_list) + 1e-4)
    print("epoch: {} train_loss: {:.3f}, train_acc: {:.3f}".format(epoch, mean_train_loss, mean_train_acc))


epoch: 0 train_loss: 0.475, train_acc: 0.582


In [ ]:
targets.shape

In [ ]:
plans

In [ ]:
for p in plans:
    print(len(p.split()))

In [ ]:
max_length = max([len(item.split()) for item in plans])

In [ ]:
what = decoder_tokenizer.tokenize(plans)

In [ ]:
what[0]

In [ ]:
for w in what:
    print(len(w))

In [ ]:
decoded_string = ""

In [ ]:
decoder_tokenizer.vocab_verse = {v:k for k,v in decoder_tokenizer.vocab.items()}

In [ ]:
for label in what[-3]:
    decoded_string += decoder_tokenizer.vocab_verse[label] + " "

In [ ]:
decoded_string

In [ ]:
torch.LongTensor(what)

In [ ]:
torch.LongTensor(decoder_tokenizer.tokenize(plans))

In [ ]:
# datetime object containing current date and time
end_time = datetime.now()
print(end_time)

In [ ]:
torch.save(model.state_dict(), "exp/{}.pth".format(end_time))

In [ ]:
#model.load_state_dict(torch.load("exp/2021-03-15 20:11:33.176086.pth"))